In [13]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from helpers.utils import CustomImageDataset
import torch
import torch.nn as nn

In [14]:
number_of_smaller_set = 636
number_of_smaller_validation_set = 24
labels = pd.read_csv('data/boneage-training-dataset.csv')
training_labels = labels.head(number_of_smaller_set)
validation_labels = labels.tail(number_of_smaller_validation_set)
validation_labels
# training_labels = training_labels

,id,boneage,male
12587,15583,132,True
12588,15584,54,True
12589,15585,162,True
12590,15586,192,True
12591,15587,50,False
12592,15588,113,False
12593,15589,84,True
12594,15591,162,True
12595,15593,180,True
12596,15594,108,True


In [17]:
transformer = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

# Using custom dataset to load images 
training_dataset = CustomImageDataset(
  root_dir='data/processed/smaller-training-set', labels=training_labels, transform=transformer
)
validation_dataset = CustomImageDataset(
  root_dir='data/processed/validation-set', labels=validation_labels, transform=transformer
)

batch_size = 12
# prepared dataloader for neural network (note it is using batch size of 3, just for this sample)
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
testing_dataloader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

In [4]:
class BoneAgeModel(nn.Module):
  def __init__(self):
    super(BoneAgeModel, self).__init__()

    self.cnn = nn.Sequential(
      nn.Conv2d(1, 32, kernel_size=2, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),

      nn.Conv2d(32, 64, kernel_size=2, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),

      nn.Flatten(),

      nn.Linear(64 * (256 // 4) * (344 // 4), 128),
      nn.ReLU(),
      nn.Linear(128, 1)
    )


  def forward(self, x):
    return self.cnn(x)

In [26]:
import torch.optim as optim

# Initialize model, loss function, and optimizer
model = BoneAgeModel()
criterion = nn.L1Loss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training Loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, ages, _, _ in training_dataloader:  # Batch size = 12
        ages = ages.float()
        optimizer.zero_grad()
        predictions = model(images).squeeze()
        loss = criterion(predictions, ages)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(training_dataloader):.2f}")


Epoch 1/15, Loss: 49.48
Epoch 2/15, Loss: 38.70
Epoch 3/15, Loss: 37.74
Epoch 4/15, Loss: 37.84
Epoch 5/15, Loss: 36.66
Epoch 6/15, Loss: 36.71
Epoch 7/15, Loss: 36.74
Epoch 8/15, Loss: 34.71
Epoch 9/15, Loss: 34.49
Epoch 10/15, Loss: 33.72
Epoch 11/15, Loss: 33.10
Epoch 12/15, Loss: 33.04
Epoch 13/15, Loss: 32.77
Epoch 14/15, Loss: 32.21
Epoch 15/15, Loss: 31.09


In [28]:
import pandas as pd
import torch
from PIL import Image

test_ids = validation_labels["id"].to_numpy()

model.eval()
test_predictions = []

with torch.no_grad():
    for image, ages, gender, img_id in testing_dataloader:
        # img_path = f"data/test/{img_id}.png"
        # image = Image.open(img_path).convert("L")  # Convert to grayscale
        # image = image.unsqueeze(0)
        
        prediction = model(image).item()  # Get predicted age
        test_predictions.append([img_id.item(), prediction])
        # true_ages.append(ages.item())

        loss = criterion(prediction, ages.view(-1).float())  # Ensure ages are float
        epoch_val_loss += loss.item()

        print(f"{img_id.item():>6} {ages.item():>8} {prediction:>8.2f}")

val_loss = epoch_val_loss / len(testing_dataloader)
print(f"\n✅ Validation Loss: {val_loss:.4f}")  # Final validation loss

# Save submission file
submission_df = pd.DataFrame(test_predictions, columns=["id", "bone_age"])
submission_df.to_csv("submission.csv", index=False)
print("📁 Submission file saved!")


AttributeError: 'float' object has no attribute 'size'

In [25]:
validation_labels

,id,boneage,male
12587,15583,132,True
12588,15584,54,True
12589,15585,162,True
12590,15586,192,True
12591,15587,50,False
12592,15588,113,False
12593,15589,84,True
12594,15591,162,True
12595,15593,180,True
12596,15594,108,True
